In [2]:
import logging
from typing import Callable, Literal, Optional, Union

import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.optimize import minimize

from bwlogger import StyleAdapter
import argparse
import logging
from pathlib import Path
from typing import Literal, Optional, Union

import numpy as np
import pandas as pd

from bwbbgdl import GoGet
from bwlogger import StyleAdapter, basic_setup
from bwutils import TODAY, Date

from utils import calculate_weights, get_available_trackers, load_trackers
from plot import plot_results

c:\Users\pcampos\Anaconda3\envs\bwgi\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


# Load Data

In [7]:
path = r"C:\Users\pcampos\OneDrive - Insper - Instituto de Ensino e Pesquisa\Asset Allocation\Projeto\Projeto Asset Allocation.xlsx"
df_base = pd.read_excel(path, index_col=0, skiprows=1, sheet_name="base_values")
df_base

,IDADDI Index,IDKAP3A Index,BZRFIMB5 Index,BZRFIB5+ Index,BCOMTR Index,IBOV Index,SPXT Index,UKXUSDT Index,HSI 1 Index,NIFTYTR Index,...,BZACCETP Index,G0O1 Index,LT09TRUU Index,B1MSBRUS Index,USDEUR Curncy,USDHKD Curncy,USDINR Curncy,USDCNY Curncy,USDJPY Curncy,USDBRL Curncy
Dates,,,,,,,,,,,,,,,,,,,,,
2000-01-03,NaN,NaN,NaN,NaN,139.9930,16930.42,2002.11,NaN,NaN,1688.44,...,5.042832,511.676,165.64,NaN,0.9763,7.7759,43.5300,8.2798,101.45,1.8190
2000-01-04,NaN,NaN,NaN,NaN,139.1780,15851.00,1925.41,NaN,NaN,1737.75,...,5.046277,511.814,166.44,NaN,0.9713,7.7775,43.5050,8.2799,103.22,1.8510
2000-01-05,NaN,NaN,NaN,NaN,138.4490,16245.14,1929.28,NaN,NaN,1692.25,...,5.049719,511.889,165.39,NaN,0.9689,7.7778,43.4900,8.2798,104.14,1.8390
2000-01-06,NaN,NaN,NaN,NaN,138.5270,16106.89,1931.51,NaN,NaN,1715.37,...,5.053163,512.037,165.89,NaN,0.9682,7.7778,43.5400,8.2797,105.23,1.8415
2000-01-07,NaN,NaN,NaN,NaN,138.1060,16309.15,1983.83,NaN,NaN,1710.81,...,5.056610,512.159,166.47,NaN,0.9713,7.7776,43.5500,8.2794,105.34,1.8270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-20,3645.809201,8601.439,9262.328,11124.580,241.9382,120445.91,11931.80,1890.58,61699.92,34873.15,...,81.385160,800.113,463.15,19.3716,0.9344,7.8044,83.6475,7.2604,158.93,5.4501
2024-06-21,3648.057215,8626.378,9275.547,11182.560,239.5644,121341.13,11913.78,1874.10,60667.52,34779.90,...,81.417120,800.472,463.15,19.4079,0.9352,7.8064,83.5400,7.2610,159.80,5.4320
2024-06-24,3649.638713,8646.920,9284.778,11190.810,241.6397,122636.96,11878.04,1893.03,60664.77,34834.33,...,81.449092,800.556,463.61,19.2372,0.9317,7.8079,83.4688,7.2594,159.62,5.3905


In [10]:
df_base_aux = pd.read_excel(path, index_col=0, usecols="A:D", sheet_name="aux")
df_base_aux

,field,alias,currency
ticker_bbg,,,
IDADDI Index,LAST_PRICE,Debentures,BRL
IDKAP3A Index,LAST_PRICE,Pre-fixados,BRL
BZRFIMB5 Index,LAST_PRICE,Inflação (<5),BRL
BZRFIB5+ Index,LAST_PRICE,Inflação (>5),BRL
BCOMTR Index,TOT_RETURN_INDEX_GROSS_DVDS,Commodities,USD
IBOV Index,TOT_RETURN_INDEX_GROSS_DVDS,Ações Brasil,BRL
SPXT Index,TOT_RETURN_INDEX_GROSS_DVDS,S&P 500,USD
UKXUSDT Index,TOT_RETURN_INDEX_GROSS_DVDS,FTSE TR Index,USD
HSI 1 Index,TOT_RETURN_INDEX_GROSS_DVDS,HIS TR Index,HKD


In [9]:
df_base.dtypes

IDADDI Index      float64
IDKAP3A Index     float64
BZRFIMB5 Index    float64
BZRFIB5+ Index    float64
BCOMTR Index      float64
IBOV Index        float64
SPXT Index        float64
UKXUSDT Index     float64
HSI 1 Index       float64
NIFTYTR Index     float64
CSIR0300 Index    float64
SX5GR Index       float64
NKYTR Index       float64
DAX Index         float64
TLT US Equity     float64
HYG US Equity     float64
BZACCETP Index    float64
G0O1 Index        float64
LT09TRUU Index    float64
B1MSBRUS Index    float64
USDEUR Curncy     float64
USDHKD Curncy     float64
USDINR Curncy     float64
USDCNY Curncy     float64
USDJPY Curncy     float64
USDBRL Curncy     float64
dtype: object